In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("spark-dataframe").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 14:21:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/16 14:21:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
filepath = "/home/ubuntu/working/spark-examples/data/titanic_train.csv"

# inferSchema 사용
titanic_sdf = spark.read.csv(filepath, inferSchema=True, header=True)
titanic_sdf.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [3]:
import pandas as pd

titanic_pdf = pd.read_csv(filepath)

# Order By

In [5]:
# Pandas에서는 sort_values
titanic_pdf.sort_values(by="Name", ascending=True).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
845,846,0,3,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.55,NaN,S
746,747,0,3,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.25,NaN,S
279,280,1,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.25,NaN,S
308,309,0,2,"Abelson, Mr. Samuel",male,30.0,1,0,P/PP 3381,24.00,NaN,C
874,875,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.00,NaN,C


In [6]:
# Pclass 오름차순, Age 내림차순

titanic_pdf.sort_values(
    by=['Pclass', 'Age'],
    ascending=[True, False]
)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,27042,30.0000,A23,S
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,PC 17754,34.6542,A5,C
493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,PC 17609,49.5042,NaN,C
745,746,0,1,"Crosby, Capt. Edward Gifford",male,70.0,1,1,WE/P 5735,71.0000,B22,S
54,55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


In [7]:
import pyspark.sql.functions as F

titanic_sdf.orderBy(
    F.col('Pclass'), ascending=False
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|         25|       0|     3|Palsson, Miss. To...|female| 8.0|    3|    1|          349909| 21.075| null|       S|
|         47|       0|     3|   Lennon, Mr. Denis|  male|null|    1|    0|          370371|   15.5| null|       Q|
|         26|       1|     3|Asplund, Mrs. Car...|female|38.0|    1|    5|          347077|31.3875| null|       S|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|            2631|  7.225| null|       C|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [8]:
# Pclass 내림차순, Age 오름차순 정렬

titanic_sdf.orderBy(
    [F.col('Pclass'), F.col('Age')]
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----+--------+
|        335|       1|     1|Frauenthal, Mrs. ...|female|null|    1|    0|PC 17611|  133.65| null|       S|
|        741|       1|     1|Hawksford, Mr. Wa...|  male|null|    0|    0|   16988|    30.0|  D45|       S|
|        352|       0|     1|Williams-Lambert,...|  male|null|    0|    0|  113510|    35.0| C128|       S|
|         56|       1|     1|   Woolner, Mr. Hugh|  male|null|    0|    0|   19947|    35.5|  C52|       S|
|        376|       1|     1|Meyer, Mrs. Edgar...|female|null|    1|    0|PC 17604| 82.1708| null|       C|
|        167|       1|     1|Chibnall, Mrs. (E...|female|null|    0|    1|  113505|    55.0|  E33|       S|
|        186|       0|     1

In [9]:
# orderBy와 동일 한 메소드인 sort

titanic_sdf.sort(
    F.col('Pclass').asc(),
    F.col('Age').desc()
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+--------+-----------+--------+
|        631|       1|     1|Barkworth, Mr. Al...|  male|80.0|    0|    0|    27042|    30.0|        A23|       S|
|        494|       0|     1|Artagaveytia, Mr....|  male|71.0|    0|    0| PC 17609| 49.5042|       null|       C|
|         97|       0|     1|Goldschmidt, Mr. ...|  male|71.0|    0|    0| PC 17754| 34.6542|         A5|       C|
|        746|       0|     1|Crosby, Capt. Edw...|  male|70.0|    1|    1|WE/P 5735|    71.0|        B22|       S|
|         55|       0|     1|Ostby, Mr. Engelh...|  male|65.0|    0|    1|   113509| 61.9792|        B30|       C|
|        457|       0|     1|Millet, Mr. Franc...|  male|65.0|    0|    0|    13

# 문제

다음 쿼리를 스파크 데이터 프레임으로 표현하세요..
1. ```SQL
    select Pclass, Name
    from ( select *
           from titanic_sdf
           order by Pclass asc, Name desc)
```

2. ```SQL
select Pclass, Name
from titanic_sdf
order by Pclass asc, Name desc
```

In [16]:
# 1
titanic_sdf.orderBy(
    [F.col('Pclass'), F.col('Name')],
    ascending=[True, False]
).select(
    F.col('Pclass'),
    F.col('Name')
).show()

+------+--------------------+
|Pclass|                Name|
+------+--------------------+
|     1|Young, Miss. Mari...|
|     1|  Wright, Mr. George|
|     1|   Woolner, Mr. Hugh|
|     1|Williams-Lambert,...|
|     1|Williams, Mr. Cha...|
|     1|Widener, Mr. Harr...|
|     1|Wick, Mrs. George...|
|     1|Wick, Miss. Mary ...|
|     1|White, Mr. Richar...|
|     1|White, Mr. Perciv...|
|     1|     Weir, Col. John|
|     1|Warren, Mrs. Fran...|
|     1|    Ward, Miss. Anna|
|     1|Walker, Mr. Willi...|
|     1|Van der hoef, Mr....|
|     1|Uruchurtu, Don. M...|
|     1|Thorne, Mrs. Gert...|
|     1|Thayer, Mrs. John...|
|     1|Thayer, Mr. John ...|
|     1|Thayer, Mr. John ...|
+------+--------------------+
only showing top 20 rows



In [17]:
# 2
titanic_sdf.select(
    F.col('Pclass'), F.col('Name')
).sort(
    F.col('Pclass').asc(),
    F.col('Name').desc()
).show()

+------+--------------------+
|Pclass|                Name|
+------+--------------------+
|     1|Young, Miss. Mari...|
|     1|  Wright, Mr. George|
|     1|   Woolner, Mr. Hugh|
|     1|Williams-Lambert,...|
|     1|Williams, Mr. Cha...|
|     1|Widener, Mr. Harr...|
|     1|Wick, Mrs. George...|
|     1|Wick, Miss. Mary ...|
|     1|White, Mr. Richar...|
|     1|White, Mr. Perciv...|
|     1|     Weir, Col. John|
|     1|Warren, Mrs. Fran...|
|     1|    Ward, Miss. Anna|
|     1|Walker, Mr. Willi...|
|     1|Van der hoef, Mr....|
|     1|Uruchurtu, Don. M...|
|     1|Thorne, Mrs. Gert...|
|     1|Thayer, Mrs. John...|
|     1|Thayer, Mr. John ...|
|     1|Thayer, Mr. John ...|
+------+--------------------+
only showing top 20 rows



# Agg
- Spark DataFrame은 DataFrame 객체에서 집계 메소드가 많이 없다.
- 집계에 관련된 메소드는 거의 대부분 `pyspark.sql.functions` 패키지에 있는 함수를 활용

In [19]:
titanic_pdf['Age'].mean()

29.69911764705882

In [20]:
titanic_pdf.mean()

PassengerId    446.000000
Survived         0.383838
Pclass           2.308642
Age             29.699118
SibSp            0.523008
Parch            0.381594
Fare            32.204208
dtype: float64

In [21]:
titanic_sdf.select(
    F.col("Age")
).mean()

AttributeError: 'DataFrame' object has no attribute 'mean'

In [22]:
titanic_pdf.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [23]:
titanic_sdf.count()

891

In [26]:
titanic_sdf_agg = titanic_sdf.select(
    F.max(F.col('Age')),
    F.min(F.col('Age')),
    F.sum(F.col('Age'))
)
titanic_sdf_agg.show()

# 항상 00 떄문에 sleclt를 활용해서 집계한다 2.2.28

+--------+--------+--------+
|max(Age)|min(Age)|sum(Age)|
+--------+--------+--------+
|    80.0|    0.42|21205.17|
+--------+--------+--------+



# Group By

In [28]:
titanic_pdf_groupBy = titanic_pdf.groupby('Pclass')
titanic_pdf_groupBy

In [30]:
titanic_pdf_groupBy.count()

# count() 집계합수를 사용해줘야지만이 출력된다

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,216,216,216,216,186,216,216,216,216,176,214
2,184,184,184,184,173,184,184,184,184,16,184
3,491,491,491,491,355,491,491,491,491,12,491


In [32]:
# 특정 컬럼에 특정 집계 연산 수행 - agg
agg_format = {
    'Age': 'max',
    'SibSp': 'sum',
    'Fare': 'mean'
}

titanic_pdf_groupBy.agg(agg_format)

,Age,SibSp,Fare
Pclass,,,
1,80.0,90,84.154687
2,70.0,74,20.662183
3,74.0,302,13.675550


In [33]:
# 스파크 SQL를 사용할때는 항상 SQL을 기준으로 생각해서 진행!!
titanic_sdf.groupby(
    F.col("Pclass")
).count().show()

'''
SELECT
    Pclass,
    count(*) as count
FROM titain_sdf
GROUPBY Pclass
'''

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+



'\nSELECT\n    Pclass,\n    count(*) as count\nFROM titain_sdf\nGROUPBY Pclass\n'

In [34]:
# count 기준 내림 차순 정렬
titanic_sdf.groupby('Pclass').count().orderBy(F.col('count'), ascending = False).show()

+------+-----+
|Pclass|count|
+------+-----+
|     3|  491|
|     1|  216|
|     2|  184|
+------+-----+



In [35]:
# count가 아닌 다른 집계 함수를 이용해 groupBy를 적용하면 pandas 데이터 프레임과 유사하게 컬럼 레벨로 집계

titanic_sdf.groupby('pclass').max().show()

# GroupBy 한다음에 모든걸을 집계하는것은 상관 없이 실행된다
# GroupBy 에서는 집계함수를 적용하지 못한것을 실행하는게 문제이다!
'''
SELECT
    Pclass
    max(PassngerId),
    max(Survived),
    max(Pclass),
    
'''

+------+----------------+-------------+-----------+--------+----------+----------+---------+
|pclass|max(PassengerId)|max(Survived)|max(Pclass)|max(Age)|max(SibSp)|max(Parch)|max(Fare)|
+------+----------------+-------------+-----------+--------+----------+----------+---------+
|     1|             890|            1|          1|    80.0|         3|         4| 512.3292|
|     3|             891|            1|          3|    74.0|         8|         6|    69.55|
|     2|             887|            1|          2|    70.0|         3|         3|     73.5|
+------+----------------+-------------+-----------+--------+----------+----------+---------+



'\nSELECT\n    Pclass\n    max(PassngerId),\n    max(Survived),\n    max(Pclass),\n    \n'

In [37]:
# 특정 컬럼에 대한 집계, 집계함수 안에서는 컬럼 속성 사용 금지
titanic_sdf.groupby('pclass').max('Age').show()

+------+--------+
|pclass|max(Age)|
+------+--------+
|     1|    80.0|
|     3|    74.0|
|     2|    70.0|
+------+--------+



In [38]:
# 여러 컬럼을 집계 대상으로 설정하기

titanic_sdf.groupby(
    [F.col('pclass'), F.col('Embarked')]
).max('Age').show()

+------+--------+--------+
|pclass|Embarked|max(Age)|
+------+--------+--------+
|     3|       C|    45.5|
|     2|       C|    36.0|
|     1|    null|    62.0|
|     1|       Q|    44.0|
|     3|       Q|    70.5|
|     2|       Q|    57.0|
|     1|       C|    71.0|
|     1|       S|    80.0|
|     3|       S|    74.0|
|     2|       S|    70.0|
+------+--------+--------+



In [39]:
# agg 함수 활용 시 F.col을 사용할 수 있다

titanic_sdf.groupby(F.col('Pclass')).agg(
    F.sum(F.col('Age')).alias('sum_age'),
    F.avg(F.col('Fare')).alias('avg_fare')    
).show()

+------+-------+------------------+
|Pclass|sum_age|          avg_fare|
+------+-------+------------------+
|     1|7111.42| 84.15468749999992|
|     3|8924.92|13.675550101832997|
|     2|5168.83| 20.66218315217391|
+------+-------+------------------+



In [40]:
spark.stop()